In [ ]:
import pandas as pd

# Load the CSV file from local storage in Colab
df = pd.read_csv("/content/Preprocessed.csv")

# Display the first few rows
df.head()


,N,P,K,temperature,humidity,ph,rainfall,label
0,90.0,42.0,43.0,20.879744,82.002744,6.502985,202.935536,20
1,85.0,58.0,41.0,21.770462,80.319644,7.038096,213.841241,20
2,60.0,55.0,44.0,23.004459,82.320763,7.840207,213.841241,20
3,74.0,35.0,40.0,26.491096,80.158363,6.980401,213.841241,20
4,78.0,42.0,42.0,20.130175,81.604873,7.628473,213.841241,20


In [ ]:
import tensorflow as tf
print("Num GPUs Available:", len(tf.config.experimental.list_physical_devices('GPU')))


Num GPUs Available: 0


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = df.drop(columns=["label"])  # Replace with actual target column name
y = df["label"]

# First, split into train (70%) and temp (30%)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.30, random_state=42)

# Split temp into validation (15%) and test (15%)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.50, random_state=42)

# Print dataset sizes
print(f"Train: {len(X_train)}, Validation: {len(X_val)}, Test: {len(X_test)}")

Train: 1540, Validation: 330, Test: 330


In [ ]:
!pip install hpbandster ConfigSpace tensorflow scikit-learn numpy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.0/131.0 kB 7.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 5.4 MB/s eta 0:00:00
  Created wheel for hpbandster: filename=hpbandster-0.7.4-py3-none-any.whl size=79986 sha256=d27fbad224cc10258cbd0f3b137c0ad3a0c17abbade890f61fb8bba53678bdab
  Stored in directory: /root/.cache/pip/wheels/fb/da/7d/af80a6b0a6898aaf2e1e93ab00cdf03251624e67f0641e9f0b
  Created wheel for ConfigSpace: filename=configspace-1.2.1-py3-none-any.whl size=115950 sha256=461c7bce529910e3b5931c70a362228651136704014c746851084c88145d9bc2
  Stored in directory: /root/.cache/pip/wheels/11/0f/36/d5027c3eeb038827889830f7efbe6a1bad8956b3eb44ab2f44
  Created 

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import ConfigSpace as CS
import ConfigSpace.hyperparameters as CSH
import hpbandster.core.nameserver as hpns
from hpbandster.optimizers import BOHB
from hpbandster.core.worker import Worker
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
import pandas as pd


# Convert data to PyTorch tensors
X_train_torch = torch.tensor(X_train.values, dtype=torch.float32)
y_train_torch = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
X_val_torch = torch.tensor(X_val.values, dtype=torch.float32)
y_val_torch = torch.tensor(y_val.values, dtype=torch.float32).view(-1, 1)
X_test_torch = torch.tensor(X_test.values, dtype=torch.float32)
y_test_torch = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

# Define PyTorch Neural Network Model
class ANN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(ANN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, 1)  # Regression output
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Define Hyperparameter Search Space for BOHB
def get_config_space():
    cs = CS.ConfigurationSpace()
    cs.add_hyperparameter(CSH.UniformIntegerHyperparameter("hidden_size", 16, 128, default_value=64))
    cs.add_hyperparameter(CSH.UniformFloatHyperparameter("learning_rate", 0.0001, 0.1, log=True, default_value=0.01))
    cs.add_hyperparameter(CSH.UniformIntegerHyperparameter("batch_size", 32, 64, default_value=32))
    return cs

# Define the BOHB Worker for ANN
class ANNWorker(Worker):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def compute(self, config, budget, **kwargs):
        model = ANN(input_size=X_train.shape[1], hidden_size=config["hidden_size"])
        criterion = nn.MSELoss()
        optimizer = optim.Adam(model.parameters(), lr=config["learning_rate"])

        # Create DataLoader
        train_dataset = torch.utils.data.TensorDataset(X_train_torch, y_train_torch)
        train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=config["batch_size"], shuffle=True)

        # Train the model
        model.train()
        for _ in range(int(budget)):  # Budget determines number of epochs
            for X_batch, y_batch in train_loader:
                optimizer.zero_grad()
                y_pred = model(X_batch)
                loss = criterion(y_pred, y_batch)
                loss.backward()
                optimizer.step()

        # Evaluate on validation set
        model.eval()
        with torch.no_grad():
            y_val_pred = model(X_val_torch).numpy()

        mae = mean_absolute_error(y_val, y_val_pred)
        return {"loss": mae, "info": config}

# BOHB Setup
NS = hpns.NameServer(run_id="ann_bohb", host="127.0.0.2", port=None)
NS.start()

worker = ANNWorker(nameserver="127.0.0.2", run_id="ann_bohb")
worker.run(background=True)

bohb = BOHB(
    configspace=get_config_space(),
    run_id="ann_bohb",
    nameserver="127.0.0.2",
    min_budget=1,  # Minimum epochs
    max_budget=20  # Maximum epochs
)

# Run BOHB Optimization
res = bohb.run(n_iterations=20)

# Shutdown BOHB
bohb.shutdown()
NS.shutdown()

# Get the best hyperparameters
best_config = res.get_incumbent_id()
best_params = res.get_id2config_mapping()[best_config]["config"]

# Train Final Model with Best Hyperparameters
best_model = ANN(input_size=X_train.shape[1], hidden_size=best_params["hidden_size"])
criterion = nn.MSELoss()
optimizer = optim.Adam(best_model.parameters(), lr=best_params["learning_rate"])

train_dataset = torch.utils.data.TensorDataset(X_train_torch, y_train_torch)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=best_params["batch_size"], shuffle=True)

best_model.train()
for _ in range(20):  # Train for 50 epochs
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        y_pred = best_model(X_batch)
        loss = criterion(y_pred, y_batch)
        loss.backward()
        optimizer.step()

# Evaluate the Final Model
best_model.eval()
with torch.no_grad():
    y_train_pred = best_model(X_train_torch).numpy()
    y_val_pred = best_model(X_val_torch).numpy()
    y_test_pred = best_model(X_test_torch).numpy()

# Performance Metrics Function
def calculate_metrics(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)
    mape = mean_absolute_percentage_error(y_true, y_pred)
    return mae, mse, rmse, r2, mape

train_metrics = calculate_metrics(y_train, y_train_pred)
val_metrics = calculate_metrics(y_val, y_val_pred)
test_metrics = calculate_metrics(y_test, y_test_pred)

# Print Results
print("Best Parameters Found by BOHB:")
print(best_params)

print("\nTraining set metrics:")
print(f"MAE: {train_metrics[0]}, MSE: {train_metrics[1]}, RMSE: {train_metrics[2]}, R²: {train_metrics[3]}, MAPE: {train_metrics[4]}")

print("\nValidation set metrics:")
print(f"MAE: {val_metrics[0]}, MSE: {val_metrics[1]}, RMSE: {val_metrics[2]}, R²: {val_metrics[3]}, MAPE: {val_metrics[4]}")

print("\nTest set metrics:")
print(f"MAE: {test_metrics[0]}, MSE: {test_metrics[1]}, RMSE: {test_metrics[2]}, R²: {test_metrics[3]}, MAPE: {test_metrics[4]}")


<ipython-input-13-af69cf7ab8c5>:39: DeprecationWarning: Please use `space.add(hyperparameter)`
  cs.add_hyperparameter(CSH.UniformIntegerHyperparameter("hidden_size", 16, 128, default_value=64))
<ipython-input-13-af69cf7ab8c5>:40: DeprecationWarning: Please use `space.add(hyperparameter)`
  cs.add_hyperparameter(CSH.UniformFloatHyperparameter("learning_rate", 0.0001, 0.1, log=True, default_value=0.01))
<ipython-input-13-af69cf7ab8c5>:41: DeprecationWarning: Please use `space.add(hyperparameter)`
  cs.add_hyperparameter(CSH.UniformIntegerHyperparameter("batch_size", 32, 64, default_value=32))


Best Parameters Found by BOHB:
{'batch_size': 60, 'hidden_size': 123, 'learning_rate': 0.0134023197578}

Training set metrics:
MAE: 2.647867202758789, MSE: 10.64143180847168, RMSE: 3.262120753202076, R²: 0.7350842952728271, MAPE: 441274972766208.0

Validation set metrics:
MAE: 2.6698899269104004, MSE: 11.263387680053711, RMSE: 3.3560970903794947, R²: 0.7389450073242188, MAPE: 659757576749056.0

Test set metrics:
MAE: 2.5746207237243652, MSE: 10.42029094696045, RMSE: 3.2280475440985144, R²: 0.7237911820411682, MAPE: 333400695111680.0
